# Air Quality in Newcastle

These graphs show the air quality data obtained from a small number of precision monitoring stations located across Newcastle, and compare them to values that have been observed during the same time-period in previous years.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime
import calendar

import calendar
cal = calendar.Calendar(0)

matplotlib.rcParams.update({
    'font.size': 13,
    'timezone': 'Europe/London'
})

## The code threw up some SettingWithCopyWarnings that I will fix at somepoint
import warnings
warnings.filterwarnings('ignore')

In [ ]:
rawDFBaseline = pd.read_pickle('../cache/baseline-airquality-airmon.pkl')
rawDFRecent = pd.read_pickle('../cache/update-airquality-airmon.pkl')

lastReading = rawDFRecent['Timestamp'].max()
lastReading = datetime.strptime(lastReading, '%Y-%m-%d %H:%M:%S')
print('Last data obtained %s' 
    % (lastReading.strftime('%d %B %Y %H:%M')))

In [ ]:
sensors = rawDFBaseline["Sensor Name"].unique().tolist()
sensorList = []

## Update sensorList to remove those with short baselines
## Set to those installed after 1st June 2019, otherwise they'll be gaps in the baseline datasets
## Also picked up some nan in Sensor Name during API call - this removes these
for s in sensors:
    temp = rawDFBaseline.copy()
    temp = temp[temp['Sensor Name']==s]
    if pd.to_datetime(temp["Timestamp"].min()) < datetime(2019, 1, 1):
        sensorList.append(s)

## Not sure how to un-hard-code these - couldn't see any 'User Friendly Names' in the metadata
sensorNames = {
        'PER_AIRMON_MONITOR1048100': 'Coast Road @ Centurion Park',
        'PER_AIRMON_MONITOR1156100': 'Angel of the North',
        'PER_AIRMON_MONITOR1056100': 'Tyne Bridge: A167 Gateshead',
        'PER_AIRMON_MONITOR914': 'Gosforth Street - Salters Road',
        'PER_AIRMON_MONITOR1155100': 'Four Lane Ends: Front Street - Benton Road',
        'PER_AIRMON_MONITOR1135100': 'Pilgrim Street - Hood Street',
        'PER_AIRMON_MONITOR915': 'Jesmond Road - Coast Road',
        'PER_AIRMON_MONITOR1157100': 'St James Boulevard - Sunderland Street'
        }

In [ ]:
## Set start date of lockdown impact analysis
## Currently set to 9th March - week prior to lockdown
startDate = '2020-03-09 00:00:00'
baselineEnd = '2020-03-08 23:45:00'

iterPlot = 1

## For each station, seasonal data from previous years have been aggregated (by hour & day) and compared to real-time data (aggregated by hour & day) since the lockdown was announced. 

* The shaded area represents a seasonal normal percentile (15th-85th) boundary for each day/hour, calculated from the current month, plus previous and following, from Jan 2017 to 13th Mar 2020.
* The dotted line represents the median or each day/hour, calculated from the same time period.
* The solid lines represent the aggregated observed data for each week since the 9th March.

Note: One station (Jesmond Road - Coast Road) does not provide Particulate Matter (PM) 2.5 or PM 10 data.

<span style="color:red;font-size:16pt;font-weight:bold"> Important! The data presented here is not corrected for weather conditions at the time of measurement. In particular, wind speed and direction can heavily influence air quality readings. Some displayed variations may be caused by environmental conditions, rather than being a direct impact of the Covid-19 societal measures. </span>





In [ ]:
for s in sensorList:
    mainTitle = sensorNames[s]
    
    ## Select Data Records for Desired Sensor
    ## Baseline
    sensorDFBaseline = rawDFBaseline[rawDFBaseline["Sensor Name"]==s]
    ## Remove readings after baseline enddate (set to 8th March so no overlap with plotted measurements)
    sensorDFBaseline  = sensorDFBaseline[sensorDFBaseline['Timestamp']<=baselineEnd]
    ## Remove suspect readings
    sensorDFBaseline = sensorDFBaseline[sensorDFBaseline["Flagged as Suspect Reading"]==False]
    ## Add columns with weekday and hour
    sensorDFBaseline.loc[:,"Weekday"] = pd.to_datetime(sensorDFBaseline['Timestamp']).dt.weekday
    sensorDFBaseline.loc[:,"Month"] = pd.to_datetime(sensorDFBaseline['Timestamp']).dt.month
    sensorDFBaseline.loc[:,"Hour"] = pd.to_datetime(sensorDFBaseline['Timestamp']).dt.hour
    ## Remove columns unnecessary for analysis
    sensorDFBaseline.drop(columns=['Flagged as Suspect Reading','Location (WKT)','Ground Height Above Sea Level',
                            'Sensor Height Above Ground','Broker Name','Third Party','Sensor Centroid Longitude',
                            'Sensor Centroid Latitude','Raw ID','Units','Timestamp','Sensor Name','<html>'], axis=1, inplace=True)

    ## Recent
    sensorDFRecent = rawDFRecent[rawDFRecent["Sensor Name"]==s]
    ## Extract after start date
    sensorDFRecent = sensorDFRecent[sensorDFRecent['Timestamp']>=startDate]
    sensorDFRecent = sensorDFRecent[sensorDFRecent["Flagged as Suspect Reading"]==False]
    sensorDFRecent.loc[:,"Weekday"] = pd.to_datetime(sensorDFRecent['Timestamp']).dt.weekday
    sensorDFRecent.loc[:,"Month"] = pd.to_datetime(sensorDFRecent['Timestamp']).dt.month
    sensorDFRecent.loc[:,"Hour"] = pd.to_datetime(sensorDFRecent['Timestamp']).dt.hour
    sensorDFRecent.loc[:,"WeekNumber"] = pd.to_datetime(sensorDFRecent['Timestamp']).dt.week
    sensorDFRecent.drop(columns=['Flagged as Suspect Reading','Location (WKT)','Ground Height Above Sea Level',
                            'Sensor Height Above Ground','Broker Name','Third Party','Sensor Centroid Longitude',
                            'Sensor Centroid Latitude','Raw ID','Sensor Name'], axis=1, inplace=True)
    
    monthsList = sensorDFRecent['Month'].unique()
    
    ## Remove months if doesn't include a Monday
    for m in reversed(monthsList):
        month = cal.monthdatescalendar(2020, m)
        firstweek = month[1]
        monthQC = str(firstweek[0]) + ' 00:00:00'
        if monthQC > sensorDFRecent['Timestamp'].max():
            monthsList.remove(m)  
    
    ## Key Variables - easy to add more plots 
    ## Wasn't sure what are 'key' variables, and didn't want to overload with information
    ## Both Baseline and Recent api calls now have everything, so theoretically adding here should be the only required change
    variables = ['NO2','PM2.5','PM10']

    ## Check if variables has been measured in recent data
    for v in variables[:]:
        if v not in sensorDFRecent['Variable'].unique():  
            variables.remove(v)
            
    plotWindows = len(variables) * len(monthsList)
    ## Set up subplots of more than 1 variable
    if plotWindows > 1:
        figHeight = plotWindows * 6.5
        fig, axs = plt.subplots(plotWindows,1, figsize=(18,figHeight))
    elif plotWindows == 1:
        fig, axs = plt.subplots(figsize=(18,6.5))
    row=0
    
    for v in variables:
        variableDFBaseline = sensorDFBaseline[sensorDFBaseline["Variable"]==v]
        variableDFRecent = sensorDFRecent[sensorDFRecent["Variable"]==v]
        
        yaxLabel = v + ' ('+ variableDFRecent['Units'].iloc[0] +')'
        
        ## Find Max Y Value for plots - makes consistent Y Axis between months
        ## Taken as 99.5%ile across all data - not max due to outliers - rounded to nearest 10
        ## Sensitive to outlier values - again, may need revisiting in future
        tempDF = variableDFBaseline.append(variableDFRecent)
        tempDF = tempDF.drop(columns=['Variable','Timestamp','Units'], axis=1)
        maxY = tempDF["Value"].quantile(.996)
        maxYRound = round(maxY/ 10.0) * 10
        
        ## Loop through months - reversed so most recent top 
        for m in reversed(monthsList):
            ## Restrict data to first Monday of month, and the Sunday after the last Monday of month 
            ## eg March 2020 = 2nd March -> 5th April 
            month = cal.monthdatescalendar(2020, m)
            firstweek = month[1]
            lastweek = month[-1]
            tempStartDate = str(firstweek[0]) + ' 00:00:00'
            tempEndDate = str(lastweek[6]) + ' 00:00:00'

            monthDFRecent = variableDFRecent[variableDFRecent['Timestamp']>=tempStartDate]
            monthDFRecent = monthDFRecent[monthDFRecent['Timestamp']<=tempEndDate]
            weekList = monthDFRecent["WeekNumber"].unique().tolist()

            ## Select Seasonal data - current month, plus preceding and following
            monthDFBaseline = variableDFBaseline[variableDFBaseline["Month"]==m]
            monthDFtemp = variableDFBaseline[variableDFBaseline["Month"]==(m-1)]
            monthDFBaseline = monthDFBaseline.append(monthDFtemp)
            monthDFtemp = variableDFBaseline[variableDFBaseline["Month"]==(m+1)]
            monthDFBaseline = monthDFBaseline.append(monthDFtemp)
            monthDFBaseline.drop(columns=['Variable'], axis=1, inplace=True)
 
            ## Aggregate Covid Data
            aggregateColumns = ['Weekday', 'Hour']
            baselineMean = monthDFBaseline.groupby(aggregateColumns, group_keys=False, as_index=False).median()
            baselineLQ = monthDFBaseline.groupby(aggregateColumns, group_keys=False, as_index=False).quantile(.15)
            baselineHQ = monthDFBaseline.groupby(aggregateColumns, group_keys=False, as_index=False).quantile(.85)

            ## Set up subplot
            if plotWindows > 1:
                plt.axes(axs[row])
            row = row+1 
            plt.title(s + ' : ' + v + ' : ' + calendar.month_name[m], fontsize=12)
            plt.xlim(0,167)
            plt.ylim(0,maxYRound)
            plt.xlabel('Day/Hour')
            plt.xticks(ticks=[0,12,24,36,48,60,72,84,96,108,120,132,144,156,168], 
                           labels=['Mon 00','Mon 12','Tues 00','Tues 12','Wed 00','Wed 12','Thurs 00','Thurs 12',
                                   'Fri 00','Fri 12','Sat 00','Sat 12','Sun 00','Sun 12'])
            plt.ylabel(yaxLabel)

            # Plot Quantiles 
            plt.fill_between(x=baselineLQ.index,y1=baselineLQ['Value'],y2=baselineHQ['Value'],
                                                            color ="#f64a8a",alpha=0.2,linewidth=0,
                                                                 label="15-85%ile: Seasonal Average")
            ## Plot Median Lines
            plt.plot(baselineMean.index,baselineMean["Value"], color = "#f64a8a",linestyle=':',alpha=0.4, 
                     label="Median: Seasonal Average")
            
            for wk in weekList: 
                ## Something strange going on with the week numbering when converting originally
                ## The str(weeks[1]-1) corrects this to match the dates downloaded
                date = pd.to_datetime('2020' + str(wk-1) + '-1', format='%Y%W-%w')
                monday = date.strftime('%b-%d')
                weekDFRecent = monthDFRecent[monthDFRecent["WeekNumber"]==wk]
        
                # Aggregate Covid Data
                aggregateColumns = ['Weekday', 'Hour']
                covidMean = weekDFRecent.groupby(aggregateColumns, group_keys=False, as_index=False).median()

                plt.plot(covidMean["Value"], label="Week of "+ monday, linewidth=1.3)          

            plt.legend(loc=0, prop={'size': 9})

    ## Different Plot Layout (Title, Figure Text) depending on Number of Panels
    ## 'y' value might need adjusting in future as more plots added 
    if len(variables) > 1:
        plt.suptitle('Air Quality - ' + mainTitle, x=0.5,y=0.91, fontsize='15', fontweight='bold')
    else:
        plt.suptitle('Air Quality - %s' % mainTitle, x=0.5,y=0.97, fontsize='15', fontweight='bold')

    ## Savefig now for consistency - Figure Text only added to 'Bottom' image for now
    timestamp = str(datetime.now().strftime("%B"))
    plt.savefig('../output/airquality-' + s.lower() + '-' + timestamp.lower() + '.png', bbox_inches='tight')

    ## Add Figure Text to Bottom Image for Dashboard
    ## 'y' value might need adjusting in future as more plots added
    if iterPlot == len(sensorList):
        plt.figtext(0.09,0.075,'Urban Observatory (https://www.urbanobservatory.ac.uk/).\n'
                        'Miles Clement <m.a.clement2@ncl.ac.uk>.', horizontalalignment='left',color='#606060',
                        fontdict={'size': 11})
     
    iterPlot = iterPlot +  1   
    plt.show()


In [ ]:
baselineCount